In [ ]:
delta='35' #set any delta value and run the entire notebook after setting your input and output paths

### Importing Libraries


In [1]:
import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import  gc
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

### Importing Weights and creating the stocklist

In [ ]:
wgt_lot_path=r'C:\Aman\Nifty correlation\Weights.csv'
weight = pd.read_csv(wgt_lot_path,parse_dates = ["date"],dayfirst = True,usecols = ["Security Symbol","Weightage","date"])

stock_list=pd.unique(weight['Security Symbol']).tolist()
print(len(stock_list))
for j in stock_list:
    # removing the below stocks as they have incorrect data
    if  j=='ADANIENT' or j=='CROMPGREAV' or j=='CROMPGREAV' or j=='GRASIM' or j=='CENTURYTEX' : 
        print(j)
        stock_list.remove(j)
for i in range(len(stock_list)):
#     print(stock_list[i])
    if stock_list[i]=='BAJAJ-AUTO':
#         print(1)
        stock_list[i]=stock_list[i].replace('-','_')

In [ ]:


############
#INPUTS
#############
data=pd.read_csv(r'C:\Aman\Nifty correlation\08-09\eq_fill_check2.csv',parse_dates = ["Nifty_Date_CE"],dayfirst = True)
output_path=r'C:\Aman\Nifty correlation\nifty 50\with missing stocks/'+delta+' delta'
wgt_lot_path=r'C:\Aman\Nifty correlation\Weights.csv'
index = "Nifty"
index_file_path = r"C:\Aman\Nifty correlation\nifty data\delta greater than 50\NIFTY-I.csv"
stfolder = r"C:\Aman\Nifty correlation\data with greeks\05-50DeltaStrikesAdded"
    
#wgt_lot_path = r"C:\Aman\aman\Dispersion Data\BN Mcap & includeok - Copy.csv"

direc = r"C:\Aman\Nifty correlation\nifty 50\with missing stocks"

#output_path = r"C:\Aman\Correlation"

index_lotsize = 1

initial_equity = 100000




lookback_period = 15
std_dev = 2

#################
#EXPIRY DATES
####################
symbols = stock_list.copy()
symbols.append(index)

exp_file_path = r"C:\Aman\Nifty correlation\expiry_dates_2022.csv"
exp_df = pd.read_csv(exp_file_path,parse_dates = ["curr_exp_date","Date"],dayfirst =True,usecols = ["curr_exp_date","Date"])
#exp_df.rename({'curr_date': 'Date'}, axis=1, inplace=True)

In [ ]:
index_file_path

In [ ]:
#Generating the current month stock data files

idx = pd.read_csv(index_file_path,parse_dates = ["Date"],dayfirst = True)
df1 = idx[idx['Ticker'].str.endswith('-I')]

index_file_path = direc + "/Nifty.csv"

df1.to_csv(index_file_path)


print(idx.shape[0])
print(df1.shape[0])
    
for i in stock_list:
    df = pd.read_csv(stfolder + '/' + i + "-I.csv",parse_dates = ["Date"],dayfirst = True)
    df1 = df[df['Ticker'].str.endswith('-I')]

    
    path = direc + '//stdata/' + i + ".csv"
    
    df1.to_csv(path)

    
    print(i,df.shape[0])
    print(i,df1.shape[0])

stfolder = direc
print(index_file_path)

In [ ]:
stfolder = direc+"/stdata"
index_file_path = direc + "/Nifty.csv"
stfolder

In [ ]:
#Separating out ATM CE and PE for stocks and Index - CURRENT month contracts
from tqdm.notebook import tqdm
stfolder = direc+"/stdata"

#from tqdm.autonotebook
idx = pd.read_csv(index_file_path,usecols = ["Date","Ticker","Option_Type","Strike",
                                                                    "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"],parse_dates = ["Date"])

idx=idx[~((idx['Date'] > '25-11-2021'))]

idx = pd.merge(idx, exp_df, on = 'Date', how = 'left')

idx_ce = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
idx_pe = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")


final = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])

final = final.T.drop_duplicates().T  #Drop Duplicates Columns

final = final.add_prefix(index +"_")
final = final.sort_values(by=index +"_"+'Date_CE')

final.to_csv(output_path+"/check-1data.csv")

#print(final)
for i in tqdm(stock_list):
    if i=='BAJAJ-AUTO':
        i=i.replace('-','_')
    print(i)
    df1 = pd.read_csv(stfolder+ '/' + i + ".csv",parse_dates = ["Date"],usecols = ["Date","Ticker","Option_Type","Strike",
                                                                    "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"])


    df1 = pd.merge(df1, exp_df, on = 'Date', how = 'left')
    
    
    df_ce = df1[(df1["Strike"] == df1["Delta_"+delta+"_Strike"]) & (df1["Option_Type"] == "CE")].add_suffix("_CE")
    df_pe = df1[(df1["Strike"] == df1["Delta_"+delta+"_Strike"]) & (df1["Option_Type"] == "PE")].add_suffix("_PE")
    print(df1.shape[0], df_ce.shape[0], df_pe.shape[0])
    temp = df_ce.merge(df_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
    temp = temp.T.drop_duplicates().T
    temp = temp.add_prefix(i +"_")
    
    final = final.merge(temp,how='left',left_on=index + '_Date_CE', right_on= i + '_Date_CE')
    
final = final.T.drop_duplicates().T
curr_final_df = final.copy()
u = curr_final_df.select_dtypes(exclude=['datetime'])
curr_final_df[u.columns] = u.fillna(0)

curr_final_df.to_csv(output_path+"/check-2data.csv")


In [ ]:
data=curr_final_df

# Adding Weights Columns

In [ ]:
weight = pd.read_csv(wgt_lot_path,parse_dates = ["date"],dayfirst = True,usecols = ["Security Symbol","Weightage","date"])
grp = weight.groupby("Security Symbol")
data[index + "_OG_Weight"] = 100
for i in stock_list:
    if i=='BAJAJ_AUTO':
        i=i.replace('_','-')
    d1 = grp.get_group(i)
    if i=='BAJAJ-AUTO':
        i=i.replace('-','_')
    d1[i + "_OG_Weight"] = d1["Weightage"]
    d1[index+ "_Date_CE"] = d1["date"]
    data = pd.merge(data,d1[[index+ "_Date_CE",i + "_OG_Weight"]],on= index+ "_Date_CE", how='left')

if data.isnull().sum().sum() != 0:
    print(data.isnull().sum().sum())
    print("NULL VALUES PRESENT ERROR")
    print("NULL VALUES PRESENT ERROR")
    print("NULL VALUES PRESENT ERROR")

display(data)

In [ ]:
data

### Adding Final Weight Columns

In [ ]:
#
for i in range(len(stock_list)):
#     print(stock_list[i])
    if stock_list[i]=='BAJAJ-AUTO':
#         print(1)
        stock_list[i]=stock_list[i].replace('-','_')

filter_col = [col for col in data if col.endswith('Weight')]
filter_col.remove(index + "_OG_Weight")

data["actual_wgt_sum"] = data[filter_col].sum(axis=1)
for i in stock_list:
    data[i + "_Final_Weight"] = data[i + "_OG_Weight"]/data["actual_wgt_sum"]
data[index + "_Final_Weight"] = 1

In [ ]:
u = data.select_dtypes(exclude=['datetime'])
data[u.columns] = u.fillna(0)

display(data)

### Adding entry and exit signals

In [ ]:
df1 = data.copy()
#df1['dispersion_entry'] = False #(df1['bb_signal'] == -1) & (df1['exit_ratio'] > 0)  & (df1['entry_ratio'].isna() == True)
#df1['dispersion_exit'] = False #(df1['bb_signal'] == 0) & (df1['entry_ratio'] > 0) & (df1['entry_ratio'].isna() == False)

df1['correlation_entry'] = np.nan
df1['correlation_exit'] = np.nan
df1['bb_signal'] = np.nan

#df1['correlation_entry'] = (df1['bb_signal'] == 1) & (df1['entry_ratio'] > 0)  & (df1['exit_ratio'].isna() == True)
#df1['correlation_exit'] = (df1['bb_signal'] == 0)  & (df1['exit_ratio'] > 0)  & (df1['exit_ratio'].isna() == False):

df1['bb_signal'] = df1.apply(lambda x: 0 if x['Nifty_Date_CE']==x['Nifty_curr_exp_date_CE'] else 1, axis=1)



for i in range(len(df1)):
    if(i==0): #for the First trade
        df1.loc[i,'correlation_entry'] = True
        df1.loc[i,'correlation_exit'] = False
        
        #If previous day was expiry current day entry is set to true
    elif (df1.loc[i-1,'Nifty_Date_CE'] == df1.loc[i-1,'Nifty_curr_exp_date_CE']):  
        df1.loc[i,'correlation_entry'] = True
        df1.loc[i,'correlation_exit'] = False
        
        #If CURRENT day IS expiry, current day Exit is set to true
    elif (df1.loc[i,'Nifty_Date_CE'] == df1.loc[i,'Nifty_curr_exp_date_CE']):
        df1.loc[i,'correlation_entry'] = False
        df1.loc[i,'correlation_exit'] = True
    
    
    else: # for The rest of the days
        df1.loc[i,'correlation_entry'] = False
        df1.loc[i,'correlation_exit'] = False

display(df1)          
    

In [ ]:
df2 = df1.copy()


df2.to_csv(output_path + '/with entry exit signals_all data.csv')

In [ ]:
df = df2[  (df2["correlation_exit"] == True) | 
               (df2["correlation_entry"] == True) ].reset_index()

df.to_csv(output_path + '/with entry exit signals.csv')
df

In [ ]:
df=pd.read_csv(output_path + '/with entry exit signals.csv')

In [ ]:
for i in range(len(stock_list)):
#     print(stock_list[i])
    if stock_list[i]=='BAJAJ-AUTO':
#         print(1)
        stock_list[i]=stock_list[i].replace('-','_')
    
#removing stocks which had their names changed
stock_list.remove('HEROHONDA')
stock_list.remove('RANBAXY')
stock_list.remove('STER')
stock_list.remove('SESAGOA')
stock_list.remove('INFOSYSTCH')
stock_list.remove('SSLT')

In [ ]:
trades = []
index='Nifty'
for i in df.index[:-1]:
    if df["bb_signal"][i] == 1:
        print(i)
        strikes_ce = {}
        strikes_pe = {}
        strikes_ce[index] = df[index + "_Strike_CE"][i]
        strikes_pe[index] = df[index + "_Strike_PE"][i]
        for j in stock_list:
            if j=='AREVAT_D' or j=='L_TFH' or j=='M_M' or j=='M_MFIN' :    
                j=j.replace('_','&')

            strikes_ce[j] = df[j+"_Strike_CE"][i]
            if j+"_Strike_PE" not in df.columns:
                     strikes_pe[j] = df[j+"_Strike_CE"][i]
            else:
                strikes_pe[j] = df[j+"_Strike_PE"][i]
        trades.append( 
                        { 
                          "entry_date": df[index + "_Date_CE"][i],
                          "exit_date" : df[index + "_Date_CE"][i+1],
                          "expiry_date" : df[index+"_curr_exp_date_CE"][i],
                          "type_of_trade": "LISS",
                          "strike_ce" : strikes_ce,
                            "strike_pe" : strikes_pe
                        }
                      )

In [ ]:

stock_list1=['ACC']
for i in tqdm(trades):
    date1 = i["entry_date"]
    date2 = i["exit_date"]

    expiry = i["expiry_date"]
    print(date1)
    print(date2)

    if expiry == date1:
        df = pd.read_csv(index_file_path,parse_dates = ["Date"])

        df = df[(df["Date"] > date1) & (df["Date"] <= date2) & (df["Strike"] == i["strike_ce"][index])]

        df_dict = {}
        df_dict[index] = nextdf

        for j in stock_list:
            if j=='AREVAT_D' or j=='L_TFH' or j=='M_M' or j=='M_MFIN' :    
                j=j.replace('_','&')
            path = stfolder + '/' + j + "-I.csv"
            df = pd.read_csv(path,parse_dates = ["Date"])
            print(df)

            df = df[((df["Date"] >= date1) & (df["Date"] <= date2)) & ((df["Strike"] == i["strike_ce"][j]) | (df["Strike"] == i["strike_pe"][j]))]

        i["dataframes"] = df_dict

    else:
        df = pd.read_csv(index_file_path,parse_dates = ["Date"])
        a=df[((df["Date"] >= date1) & (df["Date"] <= date2)) & ((df["Strike"] == i["strike_ce"][index]) ) & (df['Option_Type']=='CE')]#.add_suffix("_CE")
        b=df[((df["Date"] >= date1) & (df["Date"] <= date2)) & ((df["Strike"] == i["strike_pe"][index])  & (df['Option_Type']=='PE'))]#.add_suffix("_PE")
        df=a.append(b)
        df = pd.merge(df, exp_df, on = 'Date', how = 'left')
        df_dict = {}
        df_dict[index] = df

        for j in stock_list:
            if j=='AREVAT_D' or j=='L_TFH' or j=='M_M' or j=='M_MFIN' :    
                    j=j.replace('_','&')
            df = pd.read_csv(stfolder + '/' + j + ".csv",parse_dates = ["Date"])
            x=df[((df["Date"] >= date1) & (df["Date"] <= date2)) & ((df["Strike"] == i["strike_ce"][j]) ) & (df['Option_Type']=='CE')]#.add_suffix("_CE")
            y=df[((df["Date"] >= date1) & (df["Date"] <= date2)) & ((df["Strike"] == i["strike_pe"][j])  & (df['Option_Type']=='PE'))]#.add_suffix("_PE")
            df=x.append(y)
            
            df = pd.merge(df, exp_df, on = 'Date', how = 'left')
            
            df_dict[j] = df
        i["dataframes"] = df_dict
        

In [ ]:
#Separating CE and PE in every dataframe to format into coloumns

result = pd.DataFrame()
for trade_dict in tqdm(trades):
    df_dict = trade_dict["dataframes"]
    entry = trade_dict["entry_date"]
    exit = trade_dict["exit_date"]
    
    idx = df_dict[index][["Date","Ticker","OPT_Close","Strike","Option_Type","IV","EQ_Close","curr_exp_date"]]
    
    idx_ce = idx[(idx["Option_Type"] == "CE")].add_suffix("_CE")
    idx_pe = idx[(idx["Option_Type"] == "PE")].add_suffix("_PE")

    final = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
    final = final.add_prefix(index +"_")
    
    for i in df_dict:
        if i == index:
            continue
        print(i)
        df1 = df_dict[i][["Date","Ticker","OPT_Close","Strike","Option_Type","IV","EQ_Close","curr_exp_date"]]
        df_ce = df1[(df1["Option_Type"] == "CE")].add_suffix("_CE")
        df_pe = df1[(df1["Option_Type"] == "PE")].add_suffix("_PE")
        df_ce = df_ce.add_prefix(i +"_")
        df_pe = df_pe.add_prefix(i +"_")

        final = final.merge(df_ce,left_on=index + '_Date_CE' , right_on=i + '_Date_CE', how = 'left')
        final = final.merge(df_pe,left_on=index + '_Date_CE' , right_on= i +'_Date_PE', how = 'left').drop(columns = [i +"_"+"Option_Type_PE",i +"_"+"Option_Type_CE"])
        

    final["Trade_Type"] = trade_dict["type_of_trade"]
    
    conditions = [
    (final[index + '_Date_CE'] == entry),
    (final[index + '_Date_CE'] == exit),
    (final[index + '_Date_CE'] != exit) & (final[index + '_Date_CE'] != entry)]

    # create a list of the values we want to assign for each condition
    values = ['F', 'L', 'M']

    # create a new column and use np.select to assign values to it using our lists as arguments
    final['D_marker'] = np.select(conditions, values)
    result = result.append(final)
    
result = result.T.drop_duplicates().T

In [ ]:
result=result.drop_duplicates()#.to_csv(output_path+'/datacheck2.csv')

In [ ]:
result.to_csv(output_path+'/datacheck1.csv')

In [ ]:
data = result.copy()
display(data)
data.to_csv(output_path + '/before_weigt_add_second_time.csv')

### Adding Weight and Lot Columns Again

In [ ]:
weight = pd.read_csv(wgt_lot_path,parse_dates = ["date"],dayfirst = True,usecols = ["Security Symbol","Weightage","date"])

In [ ]:
data=pd.read_csv(output_path+'/datacheck1.csv',parse_dates=["Nifty_Date_CE"])
weight = pd.read_csv(wgt_lot_path,parse_dates = ["date"],dayfirst = True,usecols = ["Security Symbol","Weightage","date"])
grp = weight.groupby("Security Symbol")
data[index + "_OG_Weight"] = 100
data[index + "_Lotsize"] = index_lotsize
for i in tqdm(stock_list):
    data[i+'_EQ_Close_PE']=data[i+'_EQ_Close_CE']
    print(i)
    if i=='BAJAJ_AUTO':
         i=i.replace('_','-')
    d1 = grp.get_group(i)
    if i=='BAJAJ-AUTO':
         i=i.replace('-','_')
    d1[i + "_OG_Weight"] = d1["Weightage"]
    d1[i + "_Lotsize"] = 1 #d1["Lotsize"]
    d1[index+ "_Date_CE"] = d1["date"]
    data = pd.merge(data,d1[[index+ "_Date_CE",i + "_OG_Weight",i+'_Lotsize']],on= index+ "_Date_CE", how='left')
#     print(i)
#     print(data)
    
if data.isnull().sum().sum() != 0:
    print(data.isnull().sum().sum())
    print("NULL VALUES PRESENT ERROR")
    print("NULL VALUES PRESENT ERROR")
    print("NULL VALUES PRESENT ERROR")

u = data.select_dtypes(exclude=['datetime'])
data[u.columns] = u.fillna(0)

display(data)

data.to_csv(output_path + '/weigt_add_second_time1.csv')

### Adding Final Weight Columns

In [ ]:
filter_col = [col for col in data if col.endswith('Weight')]
filter_col.remove(index + "_OG_Weight")

data["actual_wgt_sum"] = data[filter_col].sum(axis=1)
for i in stock_list:
    if i=='BAJAJ-AUTO':
         i=i.replace('-','_')
    data[i + "_Final_Weight"] = data[i + "_OG_Weight"]/data["actual_wgt_sum"]
data[index + "_Final_Weight"] = 1

In [ ]:
filter_col = [col for col in data if col.endswith('Final_Weight')]
filter_col.remove(index + "_Final_Weight")
print(filter_col)

data[filter_col] = data[filter_col].replace(0,np.nan)

data["Min_Share_Value"] = data[filter_col].min(axis = 1)
data["Min_Share_Name"] = data[filter_col].idxmin(axis=1)
print(data["Min_Share_Value"][250])
print(data["Min_Share_Name"][250])

data[filter_col] = data[filter_col].replace(np.nan,0)

In [ ]:
display(data)
data.to_csv(output_path + '/before_qty_add 1.csv')

### Filling in data holes

In [ ]:
import warnings
from tqdm.notebook import tqdm
index="Nifty"
result=pd.read_csv(output_path + '/before_qty_add 1.csv')
start_dates=[]
end_dates=[]
for idi,row in result.iterrows():
    if result.loc[idi,'D_marker']=='F':
        start_dates.append(result.loc[idi,'Nifty_Date_CE'])
    if result.loc[idi,'D_marker']=='L':
        end_dates.append(result.loc[idi,'Nifty_Date_CE'])
warnings.filterwarnings("ignore")
final=pd.DataFrame()
l1=["WIPRO"]
for i in tqdm(range(len(start_dates))):

    start=start_dates[i]
    end=end_dates[i]

    temp=result[(result['Nifty_Date_CE']>=start) &
                (result['Nifty_Date_CE']<=end) ]
    temp=temp.reset_index(drop=True)
    temp.fillna(0,inplace=True)
    for stock in tqdm(stock_list):
        
        if stock=='AREVAT_D' or stock=='L_TFH'  or stock=='M_M' or stock=='M_MFIN' : 
                    stock=stock.replace('_','&')
        close=temp[stock+'_EQ_Close_CE'].iat[0]
        date=temp[index+'_Date_CE'].iat[0]
        strike_ce=0

        ticker_ce=0
        ticker_pe=0
        exp_date=0
        close_ce=0
        close_pe=0
        iv=0
        if temp[stock+'_EQ_Close_CE'].iat[0]>0:
            close=temp[stock+'_EQ_Close_CE'].iat[0]
            date=temp[index+'_Date_CE'].iat[0]

            ticker_ce=temp[stock+'_Ticker_CE'].iat[0]
            close_ce=temp[stock+'_OPT_Close_CE'].iat[0]
            iv=temp[stock+'_IV_CE'].iat[0]
            for idi,row in temp.iterrows():
                if temp.loc[idi,stock+'_EQ_Close_CE']>0:
                    close=temp.loc[idi,stock+'_EQ_Close_CE']
                    date=temp.loc[idi,index+'_Date_CE']
                    ticker_ce=temp.loc[idi,stock+'_Ticker_CE']
                    close_ce=temp.loc[idi,stock+'_OPT_Close_CE']
                    iv=temp.loc[idi,stock+'_IV_CE']

                if temp.loc[idi,stock+'_EQ_Close_CE']==0:
                    if temp.loc[idi,'D_marker']=='L':
                        print(idi,temp.loc[idi,'Nifty_Date_CE'],stock)
                    temp.loc[idi,stock+'_EQ_Close_CE']=close
                    temp.loc[idi,stock+'_Date_CE']=date
                    temp.loc[idi,stock+'_Ticker_CE']=ticker_ce
                    temp.loc[idi,stock+'_OPT_Close_CE']=close_ce
                if temp.loc[idi,stock+'_Strike_CE']==0:
                    temp.loc[idi,stock+'_IV_CE']=iv
        
        if temp[stock+'_Strike_PE'].iat[0]>0:
            iv=temp[stock+'_IV_PE'].iat[0]
            close=temp[stock+'_EQ_Close_PE'].iat[0]
            date=temp[index+'_Date_CE'].iat[0]
            ticker_pe=temp[stock+'_Ticker_PE'].iat[0]
            close_pe=temp[stock+'_OPT_Close_PE'].iat[0]
            for idi,row in temp.iterrows():
                if temp.loc[idi,stock+'_Strike_PE']>0:
                    iv=temp.loc[idi,stock+'_IV_PE']

                    close=temp.loc[idi,stock+'_EQ_Close_CE']
                    date=temp.loc[idi,index+'_Date_CE']
                    ticker_pe=temp.loc[idi,stock+'_Ticker_PE']
                    close_pe=temp.loc[idi,stock+'_OPT_Close_PE']


                if temp.loc[idi,stock+'_Strike_PE']==0:
                    if temp.loc[idi,'D_marker']=='L':
                        print(idi,temp.loc[idi,'Nifty_Date_CE'],stock)
                    temp.loc[idi,stock+'_IV_PE']=iv
                    temp.loc[idi,stock+'_EQ_Close_CE']=close
                    temp.loc[idi,stock+'_Date_CE']=date
                    temp.loc[idi,stock+'_Ticker_PE']=ticker_pe
                    temp.loc[idi,stock+'_OPT_Close_PE']=close_pe
                if temp.loc[idi,stock+'_Strike_PE']==0:
                                temp.loc[idi,stock+'_IV_PE']=iv
    final=final.append(temp)
            
                    


### Adding IV's and Implied correlation ratio

In [ ]:
final[index + "_Avg IV"] = (final[index + "_IV_CE"] + final[index + "_IV_PE"])/2
for i in tqdm(stock_list):
    final[i + "_Avg IV"] = (final[i + "_IV_CE"] + final[i + "_IV_PE"])/2

In [ ]:
final['stock_iv']=0
for i in stock_list:
    final['stock_iv'] += final[i + '_Avg IV']*final[i + "_Final_Weight"] 
    
final['implied_correl'] = (final[index + "_Avg IV"]/final['stock_iv'])


In [ ]:
final.to_csv(output_path + '/IV fill check3.csv')

### Set expiry day option closing to intrinsic value or 0.05

In [ ]:
result=pd.read_csv(output_path + '/IV fill check3.csv')
from tqdm.notebook import tqdm
stock_list2=['PNB']
for i in tqdm(stock_list):
    print(i)

    if i=='M&M':
        i='M_M'
    df=pd.read_csv(r'C:\Aman\final updated equity data/'+i+'.csv',parse_dates = ["Date"],usecols = ["Date",'Close']).reset_index(drop=True)#inplace=True)
    
    if i=='M_M':
        i='M&M'

    for idi, row in result.iterrows():
        date=result.loc[idi,'Nifty_Date_CE']
        if len(df[df['Date']==date]['Close']):
            print
            result.loc[idi,i+'_EQ_Close_CE']=df[df['Date']==date]['Close'].iat[0]
            close=float(result.loc[idi,i+'_EQ_Close_CE'])
        if  i!='TATACHEM' and i!='ADANIENT' and i!='GRASIM' and i!='CENTURYTEX': 
            if result.loc[idi,'D_marker']=='L':
                if result.loc[idi,i+ "_Ticker_CE"]!='0' and result.loc[idi,i+ "_Ticker_CE"]!=0:
                    ce_strike=float(result.loc[idi,i+ "_Ticker_CE"].replace(i,"").replace('CE-I',""))
                    pe_strike=float(result.loc[idi,i+ "_Ticker_PE"].replace(i,"").replace('PE-I',""))
                    if ce_strike>close:
                        #print(i,idi,'ce')
                        result.loc[idi,i+'_OPT_Close_CE']=0.05
                    if pe_strike<close:
                        #print(i,idi,'pe')
                        result.loc[idi,i+'_OPT_Close_PE']=0.05
                    if ce_strike<close:
                        print(idi,i,'ce',result.loc[idi,i+'_OPT_Close_CE'],close,ce_strike)
                        result.loc[idi,i+'_OPT_Close_CE']=close-ce_strike
                        print(result.loc[idi,i+'_OPT_Close_CE'])
                    if pe_strike>close:
                        print(idi,i,'pe',result.loc[idi,i+'_OPT_Close_PE'],close,pe_strike)

                        result.loc[idi,i+'_OPT_Close_PE']=pe_strike-close
                        print(result.loc[idi,i+'_OPT_Close_PE'])


In [ ]:
result.to_csv(output_path + '/eq fill with adj 1.csv')

### ADDING QTYS


In [ ]:
result = pd.read_csv(output_path + '/eq fill with adj 1.csv')
min_exp = []
for idi, row in result.iterrows():
    min_share_stock = row['Min_Share_Name'].replace("_Final_Weight","")
    value  = row[min_share_stock + "_EQ_Close_CE"]*row[min_share_stock + "_Lotsize"]
    min_exp.append(value)
    
result["MinExp"] = min_exp
#Calculating index exposure from minShareExposure
result[index + "_exp"] = (result["MinExp"]/result["Min_Share_Value"])

#calculating qty for index from exposure
result[index + "_lot"] = (result[index + "_exp"]/(result[index + "_Lotsize"]*result[index + "_EQ_Close_CE"]))
result[index + "_qty"] = result[index + "_lot"]*result[index + "_Lotsize"]

result.to_csv(output_path + '/resultt.csv')

#for stocks with includeok=0, making '_Lotsize' & '_EQ_Close_CE' non-zero

filter_col_Lotsize = [col for col in result if col.endswith('_Lotsize')]
filter_col_Lotsize.remove(index + "_Lotsize")
print(filter_col_Lotsize)
result[filter_col_Lotsize] = result[filter_col_Lotsize].replace(0,1)


filter_col_EQ_Close = [col for col in result if col.endswith('_EQ_Close_CE')]
filter_col_EQ_Close.remove(index + "_EQ_Close_CE")
print(filter_col_EQ_Close)
result[filter_col_EQ_Close] = result[filter_col_EQ_Close].replace(0,1)

result['index_exp'] = result[index + "_exp"]

result.to_csv(output_path + '/resulttcheckcheck.csv')

result['stock_exp'] = 0

for i in stock_list:
    print(i)
    if(i==min_share_stock):
        result[index + "_exp"]
        result[i + "_Final_Weight"]
        result[i + "_exp"] = result["MinExp"]
        result[i  + "_lot"] = 1
        result[i + "_qty"] = result[i + "_lot"]*result[i + "_Lotsize"]
        
        result['stock_exp'] += result[i + "_exp"]
        
    else:
        result[index + "_exp"]
        result[i + "_Final_Weight"]
        result[i + "_exp"] = (result[index + "_exp"]*result[i + "_Final_Weight"])
#         result[i  + "_lot"] = (result[i + "_exp"]/(result[i + "_Lotsize"]*result[i+ "_EQ_Close_CE"])).astype(int)
        result[i  + "_lot"] = (result[i + "_exp"]/(result[i + "_Lotsize"]*result[i+ "_EQ_Close_CE"]))
        result[i + "_qty"] = result[i + "_lot"]*result[i + "_Lotsize"]
        
        result['stock_exp'] += result[i + "_exp"]
        

In [ ]:
result.to_csv(output_path + '/qtycheckfinal 1.csv')


In [ ]:
result['total_exposure'] = (result['index_exp'] + result['stock_exp'])*2#(*2 for CE and PE)

conditions = [
    (result["Trade_Type"] == "LISS"),
    (result["Trade_Type"] == "SILS")]


#Dividing by 10, becuase we have 10x leverage

values = [result['total_exposure']/10,result['total_exposure']/10]
result['margin'] = np.select(conditions, values)

result['units'] = (initial_equity/result['margin']).astype(int)

result.to_csv(output_path+"/final2.csv")

In [ ]:
result = pd.read_csv(output_path+"/final2.csv")

In [ ]:
conditions = [
    (result["D_marker"] == "F"),
    (result["D_marker"] != "F")]

values = [result[index + "_qty"]*result['units'],np.nan]
result[index + '_qty_final'] = np.select(conditions, values)

for i in stock_list:
    values = [result[i + "_qty"]*result['units'],np.nan]
    result[i + '_qty_final'] = np.select(conditions, values)

In [ ]:
result = result.ffill()

In [ ]:
result.to_csv(output_path+"/finalfinal1.csv")

In [ ]:
display(result)

### Calculating PnL

In [ ]:
    result=pd.read_csv(r"C:\Aman\Nifty correlation\nifty 50\with missing stocks\5 delta"+"/finalfinal1.csv")
    print("Calculating PnL.....")
    print("          ", datetime.datetime.now())
    index="Nifty"
    u = result.select_dtypes(exclude=['datetime'])
    result[u.columns] = u.fillna(0)

    initial_equity=100000
    plvalue = []
    pl = 0
    transaction_cost = 0

    for idi, row in tqdm(result.iterrows()):
        result.loc[idi,'stocks_plvalue']=0

        pval = 0
        if row["D_marker"] == "F":
                        
            
            result.loc[idi, 'pl_check'] = pl + initial_equity
            result.loc[idi+1:, 'pl_check'] = np.nan
            
            (result.loc[idi ,'available_margin']) = (result.loc[idi ,'pl_check'])

            result.loc[idi , 'units_new'] = (((result.loc[idi ,'pl_check']))/result.loc[idi, 'margin'])
            result.loc[idi+1: , 'units_new'] = np.nan

            result.loc[idi, index + '_qty_final'] = result.loc[idi, index + "_qty"] * result.loc[idi, 'units_new']
            result.loc[idi+1:, index + '_qty_final'] = np.nan

            result.loc[idi, 'plvalue_' + index] = 0
            
            contract_value = result.loc[idi, index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_CE"] + result.loc[idi, index + "_OPT_Close_PE"])
            
            
            for i in stock_list:     
                    result.loc[idi, i + '_qty_final'] = result.loc[idi, i + "_qty"] * result.loc[idi, 'units_new']                    
                    result.loc[idi+1:, i + '_qty_final'] = np.nan

                    result.loc[idi, 'plvalue_' + i] = 0

                    contract_value += result.loc[idi, i + "_qty_final"]*(result.loc[idi, i + "_OPT_Close_CE"] + result.loc[idi, i + "_OPT_Close_PE"])

                    
            result.loc[idi, 'transaction_cost'] = 0
            result = result.ffill()
        else:
            if row["Trade_Type"] == "LISS":
                plvalce = result.loc[idi, index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_CE"] - result.loc[idi-1, index + "_OPT_Close_CE"])
                plvalpe = result.loc[idi, index + "_qty_final"]*(result.loc[idi, index + "_OPT_Close_PE"] - result.loc[idi-1, index + "_OPT_Close_PE"])
                pval = plvalce + plvalpe

                result.loc[idi, 'plvalue_' + index] = plvalce + plvalpe

                for i in stock_list:
                    
                        plvalce = result.loc[idi, i + "_qty_final"]*(result.loc[idi-1, i + "_OPT_Close_CE"] - result.loc[idi, i + "_OPT_Close_CE"])
                        plvalpe = result.loc[idi, i + "_qty_final"]*(result.loc[idi-1, i + "_OPT_Close_PE"] - result.loc[idi, i + "_OPT_Close_PE"])
                        pval += plvalce + plvalpe
                        result.loc[idi,'stocks_plvalue']=result.loc[idi,'stocks_plvalue']+plvalce + plvalpe 

                        result.loc[idi, 'plvalue_' + i] = plvalce + plvalpe 





            pl += pval


        if row['D_marker'] == "L":

            contract_value += result.loc[idi-1, index + "_qty_final"]*(result.loc[idi-1, index + "_OPT_Close_CE"] + result.loc[idi-1, index + "_OPT_Close_PE"])

            for i in stock_list:                                                                    
                contract_value += result.loc[idi-1, i + "_qty_final"]*(result.loc[idi-1, i + "_OPT_Close_CE"] + result.loc[idi-1, i + "_OPT_Close_PE"])

            transaction_cost = contract_value*0.005
            pl = pl - transaction_cost
            result.loc[idi, 'transaction_cost'] = transaction_cost

        plvalue.append(pval)

    result["Daily_PL"] = plvalue

    #result.to_csv(output_path+"\pl_final4.csv")


    result.loc[0,'Daily_PL'] = initial_equity
    result["PortfolioValue"] = (result["Daily_PL"] - result["transaction_cost"]).cumsum()
    result.loc[0,'Daily_PL'] = 0

    display(result)
    #result.to_csv(r'C:\Aman\Nifty correlation\outputs/final4.1__.csv')
    
    print("done")

In [ ]:
result.to_csv(output_path+"/pl1 "+delta+".csv")

## Plotting the equity curve

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(28,17))
plt.title('Equity Curve')
plt.xlabel('Days')
plt.ylabel('PortfolioValue')
plt.plot((result["Nifty_Date_CE"]).astype('str'),result['PortfolioValue'], c = 'blue')

plt.xticks(rotation=45)

plt.legend()
plt.show()

### Generating the Checksheet

In [ ]:
#result=pd.read_csv(r'C:\Aman\Nifty correlation\07-09/pl_Test1.csv')
finallist = pd.DataFrame()
index='Nifty'
temp1 = pd.DataFrame()
tradelist = result[ (result["D_marker"] == "F") | (result["D_marker"] == "L") ]

if list(tradelist.iloc[[-1]]["D_marker"])[0] == "F":
    tradelist.drop(tradelist.tail(1).index,inplace=True)
else:
    tradelist = tradelist

tr1 = tradelist[tradelist["D_marker"]=="F"].reset_index().drop(columns = ["index"])
tr2 = tradelist[tradelist["D_marker"]=="L"].reset_index().drop(columns = ["index"])
mdict = {}
print(tr1.shape[0])
if tr1.shape[0] == tr2.shape[0]:
    for idx in tr1.index:
        
        

        for i in stock_list:
            mdict = {}
            #print(i)
            for j in ["CE","PE"]:
                mdict['Name']=i#.append(i)
                mdict["Entry_Date"] = tr1[index+ "_Date_CE"][idx]
                mdict["Exit_Date"] = tr2[index+ "_Date_CE"][idx]
                mdict["EQ_at_entry"] = tr1[i+ "_EQ_Close_CE"][idx]
                mdict["EQ_at_exit"] = tr2[i+ "_EQ_Close_CE"][idx]
                
                if tr1[i+ "_Ticker_CE"][idx]!='0' and tr1[i+ "_Ticker_CE"][idx]:
                    mdict["Ticker_CE"] = tr1[i+ "_Ticker_CE"][idx].replace(i,"").replace('CE-I',"")
                    mdict["_Ticker_PE"] = tr2[i+ "_Ticker_PE"][idx].replace(i,"").replace('PE-I',"")
                else:
                    mdict["Ticker_CE"] = 0
                    mdict["_Ticker_PE"] = 0
    

                
                mdict[ "_Entry_Price_" + j] = tr1[i + "_OPT_Close_" + j][idx]
                
                mdict["_Exit_Price_" + j] = tr2[i + "_OPT_Close_" + j][idx]
                mdict["_Qty_final"] = tr1[i + "_qty_final"][idx]
                mdict["Exposure"] = tr1[i + "_exp"][idx]
                mdict["P/L"] = tr1[ "plvalue_"+i][idx]
            temp = pd.DataFrame([mdict])
            temp1=temp1.append(temp)
        #display(temp)
    finallist = finallist.append(temp1)
else:
    print("Error, Tradelist not proper")

finallist.reset_index().drop(columns = ["index"])


In [ ]:
finallist.to_csv(output_path+'/final check2.csv')

In [ ]:
### Calculate CAGR 
result=pd.read_csv(r"C:\Aman\Nifty correlation\nifty 50\with missing stocks\45 delta"+"/pl1 45.csv")
df = result.copy()
from pyxirr import xirr

firstValue = round(df.loc[0, 'PortfolioValue'],4)
lastValue = round(df.loc[df.index[-1], 'PortfolioValue'],4)

firstDate = (df.loc[0, 'Nifty_Date_CE'])
lastDate = (df.loc[df.index[-1], 'Nifty_Date_CE'])

dates = [firstDate, lastDate]
amounts = [-firstValue, lastValue]

xirr = xirr(dates, amounts)

print(xirr)
### Calculate Daily Drawdown

Roll_Max = round(df['PortfolioValue'].expanding().max(), 2)
Daily_Drawdown = (round(df['PortfolioValue'], 2)/Roll_Max) - 1.0


#Roll_Max[0:50], Daily_Drawdown[0:50]

df['Daily_Drawdown'] = Daily_Drawdown * 100

print("max dd", min(df['Daily_Drawdown']))
print(100*xirr/-min(df['Daily_Drawdown']))
df.to_csv(output_path + '/DailyDrawdown.csv', index=False)

### Monthly PNL Percentage 

df['Nifty_Date_CE'] = pd.to_datetime(df['Nifty_Date_CE'], dayfirst=True)

df['Year'] = pd.DatetimeIndex(df['Nifty_Date_CE']).year
df['Month'] = pd.DatetimeIndex(df['Nifty_Date_CE']).month
#df.to_csv(output_path + '/df.csv', index=False)

i = 0
dfg = df.groupby(['Year', 'Month'])
for name, group in dfg:
    #print(name)
    if i == 0: 
        firstValue = group['PortfolioValue'].iloc[0]
        i = 1
    else:
        firstValue = lastValue

    lastValue = group['PortfolioValue'].loc[group.index[-1]]
    change = lastValue/firstValue - 1
    df.loc[group.index, 'Change_%_Monthly'] = round(change * 100, 2)
    
i = 0
dfg = df.groupby(['Year'])
for name, group in dfg:
    #print(name)
    if i == 0: 
        firstValue = group['PortfolioValue'].iloc[0]
        i = 1
    else:
        firstValue = lastValue

    lastValue = group['PortfolioValue'].loc[group.index[-1]]
    change = lastValue/firstValue - 1
    df.loc[group.index, 'Change_%_Yearly'] = change * 100
    
df

df1 = df[['Year', 'Month', 'Change_%_Monthly', 'Change_%_Yearly']]
df1 = df1.drop_duplicates()
df1

pivotTable = df1.pivot_table(values ='Change_%_Monthly', index =['Year', 'Change_%_Yearly'],
                         columns =['Month'])
pivotTable.columns = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
                      'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

pivotTable.index

def _color_red_or_green(val):
    color = '#EE0000' if val < 0 else '#00EE00'
    return 'background-color: %s' % color
pivotTable=pivotTable.style.applymap(_color_red_or_green)
display(pivotTable)

import pandas as pd
import openpyxl as op
import numpy as np


from openpyxl import Workbook, worksheet, load_workbook
pivotTable.to_excel(output_path+'/t1.xlsx')#.to_csv(r'C:\Aman\Nifty correlation\nifty 50\19-09\pl 50 table.csv')

In [ ]:
output_path